<h1> DRL Robot Navigation with ROS2 - Tutorial </h1>
<h4> Written by Mark Wanis & Jaran Mann, Fall 2025 </h4>

<h2> <b>Introduction</b> </h2>
<p>The goal of this project is developing a strong foundation in being able to manipulate every aspect of the Turtlebot. Once a solid foundation is developed, it aims to use machine learning to navigate the Turtlebot around obsticles towards its destination in real time.  We aim to use online simulations, via Gazebo, prior to implementing machine learning into the physical robots.  Through proper testing and optimizing, the new functions will allow for a more advanced level of artificial intelligence in the turtlebots. </p>

<h2><b>Turtlebot/ROS Basic Operations</b></h2>
<p>Always refer to this website if there are any questions regarding setups and operations: <a href="https://emanual.robotis.com/docs/en/platform/turtlebot3/quick-start/">https://emanual.robotis.com/docs/en/platform/turtlebot3/quick-start/</a></p>
<h3>Creating Workspaces</h3>
<h4>Create a new workspace:</h4>
<pre><code>mkdir -p ~/colcon_ws/src</code></pre>
<h4>Add a project to the workspace:</h4>
<pre><code>ros2 pkg create --build-type ament_python turtlelab5
cd turtlelab5/turtlelab5
wget https://raw.githubusercontent.com/ROBOTIS-GIT/turtlebot3/humble-devel/turtlebot3_example/turtlebot3_example/turtlebot3_obstacle_detection/main.py
wget https://raw.githubusercontent.com/ROBOTIS-GIT/turtlebot3/humble-devel/turtlebot3_example/turtlebot3_example/turtlebot3_obstacle_detection/turtlebot3_obstacle_detection.py</code></pre>

<h3>Connecting to/configuring Turtlebots</h3>
<p>Here are some main commands and information that will be needed.</p>
<p style="text-indent: 2em;">The address of Turtlebot 53: MAC:9D / IP:192.168.2.2 (the IP address changes)</p>

<p>To launch the Turtlebot, type the following commands in the terminal:</p>
<pre><code>ssh ubuntu@{IP_ADDRESS}
ros2 launch turtlebot3_bringup robot.launch.py
</code></pre>

<p>After launching the Turtlebot, open a new terminal and run these commands to connect with the Turtlebot:</p>
<pre><code>export ROS_DOMAIN_ID={turtlebot ID} (our turtlebot ID was 51)
export RMW_IMPLEMENTATION=rmw_fastrtps_cpp
export TURTLEBOT3_MODEL=burger
</code></pre>

<p>To build or update the workspace:</p>
<pre><code>cd to back to colcon_ws
colcon build --packages-select turtlelab5
source install/setup.bash
</code></pre>

<p>Programs to run:</p>
<pre><code>ros2 run turtlebot3_teleop teleop_keyboard
ros2 run turtlelab5 go
</code></pre>

<p>Before running any research/testing code, it is a good habit to run the teleop keyboard first.  This is to ensure that you are actually connected to the turtlebot.</p>

<h2><b>Basics of Turtlebot Publishers and Subscribers</b></h2>
<p>Here are the basics of getting comfortable with manipulating both the inputs and outputs of the robot: <a href="researchproject.py">researchproject.py</a></p>
<h3><b>Publishers:</b></h3>
<ul>
    <li>Command Velocity</li>
    <ul>
        <li>Linear Velocity</li>
        <li>Angular Velocity</li>
    </ul>
    
</ul>
<h3><b>Subscribers:</b></h3>
<ul>
    <li>Lidar</li>
    <li>Odometry</li>
    <ul>
        <li>Position </li>
    </ul>
    <li>JointState</li>
    <ul>
        <li>Velocity </li>
    </ul>
    <li>Gyroscope</li>
    <ul>
        <li>Heading</li>
    </ul>
</ul>
<h3><b>Publisher and Subscriber Callbacks:</b></h3>
<ul>
    <li>Velocity Callback</li>
</ul>

In [ ]:
# Updates and publishes the velocity to the robot
def velocity_callback(self):
    # Creating the data object
    twist = Twist()
    
    # For the first 2 seconds, the robot will not move
    if self.current_time <= 2:
        # linear.x is what is used for the robot to move forward
        # if the time is less than or equal to 2, then set linear.x to the current linear velocity
        twist.linear.x = 0.0
        # angular.z is what is used to turn the robot
        # if the time is less than or equal to 2, then set angular.z to the current angular velocity
        twist.angular.z = 0.0

    # After 2 seconds, the robot will move
    else:
        # if the time is greater than 2, then set linear.x and angular.z to 0.1
        twist.linear.x = self.velocity.linear
        twist.angular.z = self.velocity.angular
    
    # This is how the message is published to the robot
    self.cmd_vel_pub.publish(twist)

    # Save expected velocity data
    if self.save_data:
        with open("expectedVelocity.csv", mode='a', newline='') as file:
            writer=csv.writer(file)
            if self.current_time <= 2:
                writer.writerow([self.current_time, 0, 0]) 
            else:
                writer.writerow([self.current_time, self.velocity.linear, self.velocity.angular]) 
    
    self.get_logger().info(f"Expected Velocity: {self.velocity}")

<ul>
    <li>Scan Callback</li>
</ul>

In [ ]:
# Every time the laser scanner data has been received from the turtlebot
def scan_callback(self, msg):
    # Set the scan_ranges list to be the current ranges from the scanner
    self.scan_ranges = msg.ranges
    # Set the angle between the scans
    self.angle_increment = msg.angle_increment

    # Set the time taken for a full scan
    self.scan_time = self.get_time(msg.header.stamp)

    self.scannerReceived = True
    
    # Print values
    self.get_logger().info(f"Scan ranges: {self.scan_ranges[:5]} ... {self.scan_ranges[-5:]}")
    self.get_logger().info(f"\tTime stamp: {self.scan_time:.3f} s")

<ul>
    <li>Odom Callback</li>
</ul>

In [11]:
# Every time the odometry data has been received from the turtlebot
def odom_callback(self, msg):
    # The first time, set the initial values
    if self.odomReceived == False:
        self.location_init.x = msg.pose.pose.position.x
        self.location_init.y = msg.pose.pose.position.y
        self.odomReceived = True
    
    # Determine the x and y location from the difference of the odometry and initial values
    # This way, the values will always start at (0,0)
    self.location.x = msg.pose.pose.position.x - self.location_init.x
    self.location.y = msg.pose.pose.position.y - self.location_init.y

    # Updates the actual velocity found from the odometry
    #self.velocity_odom.linear = msg.twist.twist.linear.x
    #self.velocity_odom.angular = msg.twist.twist.angular.z

    # Set the time taken for the odometry reading
    self.odom_time = self.get_time(msg.header.stamp)
    
    # Print values
    self.get_logger().info(f"Location: {self.location}")
    #self.get_logger().info(f"Actual Velocity: {self.velocity_odom}")
    self.get_logger().info(f"\tTime stamp: {self.odom_time:.3f} s")

<ul>
    <li>IMU Callback</li>
</ul>

In [8]:
# Every time the inertial subscriber is received from the turtlebot
def imu_callback(self, msg):
    #get the angle information in quaternion format
    qx = msg.orientation.x
    qy = msg.orientation.y
    qz = msg.orientation.z
    qw = msg.orientation.w
    #convert to Euler angle (we only need z direction)
    t3 = +2.0 * (qw * qz + qx * qy)
    t4 = +1.0 - 2.0 * (qy * qy + qz * qz)
    self.heading = math.atan2(t3, t4)/np.pi #Euler angle in pi-radians (-1 to 1?!)

    if self.heading >= 0:
        self.heading *= math.pi
    else:
        self.heading = self.heading * math.pi + 2.0 * math.pi
        
    if not self.imuReceived:
        self.heading_init = self.heading
        self.imuReceived = True

    # Set the time taken for the imu reading
    self.imu_time = self.get_time(msg.header.stamp)
    
    # Print values
    self.get_logger().info(f"Heading: {self.heading:.3f} rad")
    self.get_logger().info(f"\tTime stamp: {self.imu_time:.3f} s")

<ul>
    <li>Joint State Callback</li>
</ul>

In [ ]:
# Every time the JointState subscriber is received from the turtlebot
def joint_state_callback(self, msg: JointState):
    if len(msg.velocity) != 2:
        self.get_logger().warn("JointState missing wheel velocities")
        return
        
    if not self.encReceived:
        self.encReceived = True

    v_l = msg.velocity[0]  # Grabs the left wheel velocity
    v_r = msg.velocity[1]  # Grabs th right wheel velocity

    # Updates the linear and angular velocities via the encoders
    self.velocity_enc.linear = (v_r + v_l) / 2.0
    self.velocity_enc.angular = (v_r - v_l) / self.WHEEL_BASE
    
    # Set the time taken for the encoder reading
    self.enc_time = self.get_time(msg.header.stamp)
    
    # Save actual velocity data
    if self.save_data:
        with open("actualVelocity.csv", mode='a', newline='') as file:
            writer=csv.writer(file)
            writer.writerow([self.enc_time, self.velocity_enc.linear, self.velocity_enc.angular]) 
    
    # Print values
    self.get_logger().info(f"Actual Velocity: {self.velocity_enc}")
    self.get_logger().info(f"\tTime stamp: {self.enc_time:.3f} s")

<h2><b>Package Installations</b></h2>
<p>Here are the packages we installed, either for convenience or because the project needed them.</p>
<h3>Optional Packages:</h3>
<ul>
  <li>
    <p><b>VS Code</b></p>
    <pre><code>sudo snap install code --classic
    </code></pre>
  </li>
  <li>
    <p><b>Jupyter Notebook</b></p>
    <pre><code>sudo apt update
pip install jupyter
    </code></pre>
  </li>
  <li>
    <p><b>GitHub Desktop</b></p>
    <pre><code>wget -qO - https://mirror.mwt.me/shiftkey-desktop/gpgkey | gpg --dearmor | sudo tee /etc/apt/keyrings/mwt-desktop.gpg > /dev/null
echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/mwt-desktop.gpg] https://mirror.mwt.me/shiftkey-desktop/deb/ any main" | sudo tee /etc/apt/sources.list.d/mwt-desktop.list
sudo apt update
sudo apt install github-desktop
    </code></pre>
  </li>
</ul>

<h3>Gazebo Dependencies:</h3>
<pre><code>sudo apt update
sudo apt install -y \
  ros-humble-rclpy \
  ros-humble-nav-msgs \
  ros-humble-geometry-msgs \
  ros-humble-sensor-msgs \
  ros-humble-gazebo-ros-pkgs \
  ros-humble-cartographer \
  ros-humble-cartographer-ros \ 
  ros-humble-turtlebot3-cartographer</pre></code>

<h3>Python Dependencies:</h3>
<pre><code>pip install --upgrade pip setuptools wheel
pip install \
  numpy==1.26.4 \
  torch torchvision torchaudio \
  tensorboard \
  squaternion \
  stable-baselines3 \
  gym \
  matplotlib \
  pandas</pre></code>

<h2><b>Getting a Virtual Map Using Cartographer and Gazebo</b></h2>
<p>Follow these steps to set up the environment and generate a virtual map:</p>
<h3><b>Basics of Cartogapher and Gazebo</b></h3>
<ol>
  <li>
    <p>(In a terminal) Open tele-op:</p>
    <pre><code>export TURTLEBOT3_MODEL=burger
ros2 run turtlebot3_teleop teleop_keyboard
     </code></pre>
  </li>

  <li>
    <p>(In a different terminal) Open Gazebo:</p>
    <pre><code>export TURTLEBOT3_MODEL=burger
ros2 launch turtlebot3_gazebo turtlebot3_world.launch.py
     </code></pre>
  </li>

  <li>
    <p>(In a different terminal) Open Cartographer:</p>
    <pre><code>export ROS_DOMAIN_ID={turtlebot ID}
export TURTLEBOT3_MODEL=burger
ros2 launch turtlebot3_cartographer cartographer.launch.py use_sim_time:=True
     </code></pre>
  </li>
</ol>

<h2><b>Training and Testing the Virtual Turtlebot</b></h2>
<p>To train the turtlebot, we used someone else's code that allowed the turtlebot to learn how to navigate. Here are links we used to get the program to run: </p>
<h3><b>Project Links</b></h3>
<ul>
    <li>Project folder: <a href=https://drive.google.com/drive/folders/1JJHC59IyypIF9JXdzzF1K_zQiq_fo0Dv>https://drive.google.com/drive/folders/1JJHC59IyypIF9JXdzzF1K_zQiq_fo0Dv</a></li>
    <li>Explanation video: <a href=https://www.youtube.com/watch?v=KEObIB7RbH0>https://www.youtube.com/watch?v=KEObIB7RbH0</a></li>
    <li>Trained policy: <a href=https://drive.google.com/file/d/1QvO2NmUejIu2bj_4t29wrM8-eu9SRc6w/view>https://drive.google.com/file/d/1QvO2NmUejIu2bj_4t29wrM8-eu9SRc6w/view</a></li>
</ul>
<h3><b>How to view the trained policy using simulations:</b></h3>
<ol>
    <li>Download the project folder and place it into the src of your workspace (i.e. colcon_ws/src)</li>
    <li>Copy td3_velodyne_actor.pth from the trained policy folder (located in trained_policy/pytorch_models) and replace the old td3_velodyne_actor.pth file (located in DRL_robot_navigation_ros/src/td3/scripts/pytorch_models)</li>
    <li>cd into your project folder (i.e. cd colcon_ws/src/DRL_robot_navigation_ros)</li>
    <li>Build and source the project: </li><br>
    <pre><code>colcon build
source DRL_robot_navigation_ros/install/setup.bash
    </code></pre>
    <li>Run the test simulation (if training replace test_simulation with training_simulation):</li><br>
    <pre><code>ros2 launch td3 test_simulation.launch.py</code></pre>
</ol>

In [1]:
<h2><b> Displaying RViz Lidar </b></h2>
    <pre><code> ROS_DOMAIN_ID={turtlebot ID}</code></pre>
    <pre><code> ros2 launch turtlebot3_bringup rviz2.launch.py </code></pre>

SyntaxError: invalid syntax (3751810429.py, line 1)

<h2><b>Applying the Trained Policy onto the Physical Turtlebot</b></h2>
<ol>
    <li>Once the trained policy is downloaded as a .pth file, copy the <a href="test_velodyne_node.py">test_velodyne_node.py</a></li>
    <li>Edit the code to work for an actual turtlebot. The resulting should look like <a href="test_turtlebot_node.py">test_turtlebot_node.py</a></li>
    <li>Additionally, copy the test_simulation.launch and change it to look like <a href="test_turtlebot.launch.py">test_turtlebot.launch.py</a></li>
</ol>